In [99]:
import os
from collections import Counter

import psycopg2

In [4]:
racine_source = '/home/michel/legi_plat/'

In [29]:
categories = [
    'TNC_en_vigueur_JORF',
    'TNC_en_vigueur_LEGI',
    'code_en_vigueur_LEGI',
    'TNC_non_vigueur_JORF',
    'TNC_non_vigueur_LEGI',
    'code_non_vigueur_LEGI',
]

## cid présents dans plusieurs catégories

In [41]:
liste_cid_par_categorie = {}
set_cid_par_categorie = {}
for categorie in categories:
    liste_cid_par_categorie[categorie] = list(os.listdir(os.path.join(racine_source, categorie)))
    set_cid_par_categorie[categorie] = set(liste_cid_par_categorie[categorie])

In [42]:
nb_cid_par_categorie = {cat:len(l) for cat, l in liste_cid_par_categorie.items()}

In [43]:
nb_cid_par_categorie, sum(nb_cid_par_categorie.values())

({'TNC_en_vigueur_JORF': 76944,
  'TNC_en_vigueur_LEGI': 1671,
  'TNC_non_vigueur_JORF': 17434,
  'TNC_non_vigueur_LEGI': 1771,
  'code_en_vigueur_LEGI': 75,
  'code_non_vigueur_LEGI': 31},
 97926)

In [44]:
liste_cid = [cid for categorie in categories for cid in liste_cid_par_categorie[categorie]]

In [45]:
len(liste_cid)

97926

In [46]:
set_cid = set(liste_cid)

In [47]:
len(set_cid)

97040

In [48]:
print('{:.2f}% des cid apparaissent dans plusieurs catégories'.format((len(liste_cid) - len(set_cid))/len(liste_cid) * 100))

0.90% des cid apparaissent dans plusieurs catégories


In [49]:
categorie_par_cid = {}
liste_cid_unique = list(set_cid)
for cid in liste_cid_unique:
    categorie_par_cid[cid] = [categorie for categorie in categories if cid in set_cid_par_categorie[categorie]]

In [51]:
nb_categorie_par_cid = {cid: len(categorie_par_cid[cid]) for cid in liste_cid_unique}

In [55]:
Counter(nb_categorie_par_cid.values())

Counter({1: 96154, 2: 886})

In [73]:
liste_cid_couple = [cid for cid in liste_cid_unique if len(categorie_par_cid[cid])>1]

In [74]:
for cid in liste_cid_couple:
    print(cid)

JORFTEXT000030284731
JORFTEXT000031601403
JORFTEXT000031585569
JORFTEXT000000278426
JORFTEXT000000702306
JORFTEXT000030540400
JORFTEXT000000421440
JORFTEXT000022070186
JORFTEXT000000272012
JORFTEXT000026263780
JORFTEXT000028134031
JORFTEXT000032037039
JORFTEXT000023728079
JORFTEXT000024873712
JORFTEXT000023894109
JORFTEXT000022481510
JORFTEXT000027731498
JORFTEXT000031057186
JORFTEXT000000379918
JORFTEXT000029590341
JORFTEXT000032251629
JORFTEXT000000710158
JORFTEXT000030378334
JORFTEXT000030395369
JORFTEXT000030711236
JORFTEXT000026380878
JORFTEXT000031258493
JORFTEXT000000626191
JORFTEXT000029884648
JORFTEXT000030391208
JORFTEXT000031279051
JORFTEXT000000820515
JORFTEXT000031313461
JORFTEXT000000695940
JORFTEXT000000298982
JORFTEXT000029257515
LEGITEXT000006072846
JORFTEXT000031602124
JORFTEXT000000584510
JORFTEXT000000819675
JORFTEXT000020725884
JORFTEXT000031602100
JORFTEXT000029175801
JORFTEXT000000797557
JORFTEXT000029913593
JORFTEXT000031679637
JORFTEXT000000791824
JORFTEXT00002

Il y a 886 cid qui apparaissent dans 2 catégories.

In [68]:
couples_categorie = [categorie_par_cid[cid] for cid in liste_cid_unique if len(categorie_par_cid[cid])>1]

In [69]:
couples_categorie = [sorted(c) for c in couples_categorie]

In [71]:
couples_categorie = [(c[0], c[1]) for c in couples_categorie]

In [72]:
Counter(couples_categorie)

Counter({('TNC_en_vigueur_JORF', 'TNC_non_vigueur_JORF'): 881,
         ('TNC_en_vigueur_LEGI', 'TNC_non_vigueur_LEGI'): 2,
         ('code_en_vigueur_LEGI', 'code_non_vigueur_LEGI'): 3})

## Comparaison struct et version

In [96]:
for categorie in categories:
    liste_cid = list(os.listdir(os.path.join(racine_source, categorie)))

    etat_par_cid = {}
    for cid in liste_cid:
        liste_struct = sorted(list(os.listdir(os.path.join(racine_source, categorie, cid, 'struct'))))
        liste_version = sorted(list(os.listdir(os.path.join(racine_source, categorie, cid, 'version'))))
        etat_par_cid[cid] = (liste_struct == liste_version)
        nb_struct_par_cid[cid] = len(liste_struct)
        
    liste_cid_diff = [cid for cid in liste_cid if etat_par_cid[cid] == False]
    
    nb_struct_par_cid = {cid: nb_struct_par_cid[cid] if etat_par_cid[cid] else -1 for cid in liste_cid}
    compteur_taille = dict(Counter(nb_struct_par_cid.values()))
    liste_cid_zero = [cid for cid in liste_cid if nb_struct_par_cid[cid] == 0]
    
    nb_cid = len(liste_cid)
    nb_cid_diff = len(liste_cid_diff)
    nb_cid_zero = len(liste_cid_zero)    
    
    print('Catégorie {} :'.format(categorie))
    print('    {} / {} ont des sous-dossiers différents.'.format(nb_cid_diff, nb_cid))
    print('    -> ' + ', '.join(liste_cid_diff))
    print('    Répartition des tailles de sous-dossiers : {}'.format(str(compteur_taille)))
    print("    {} / {} ont des sous-dossiers vides.".format(nb_cid_zero, nb_cid))
    print('    -> ' + ', '.join(liste_cid_zero))
    print()


Catégorie TNC_en_vigueur_JORF :
    0 / 76944 ont des sous-dossiers différents.
    -> 
    Répartition des tailles de sous-dossiers : {0: 386, 1: 74947, 2: 1481, 3: 115, 4: 9, 5: 4, 6: 1, 8: 1}
    386 / 76944 ont des sous-dossiers vides.
    -> JORFTEXT000030802198, JORFTEXT000023474689, JORFTEXT000023727979, JORFTEXT000000434015, JORFTEXT000000456880, JORFTEXT000027994082, JORFTEXT000000697479, JORFTEXT000000782766, JORFTEXT000000349194, JORFTEXT000030802340, JORFTEXT000019325632, JORFTEXT000030126633, JORFTEXT000018692779, JORFTEXT000000194300, JORFTEXT000000591462, JORFTEXT000021504554, JORFTEXT000024829782, JORFTEXT000000248517, JORFTEXT000028279710, JORFTEXT000000504748, JORFTEXT000030194229, JORFTEXT000000656981, JORFTEXT000022765140, JORFTEXT000000712277, JORFTEXT000017754536, JORFTEXT000022271835, JORFTEXT000020692329, JORFTEXT000018700191, JORFTEXT000000695940, JORFTEXT000029147629, JORFTEXT000000328953, JORFTEXT000000601432, JORFTEXT000030707288, JORFTEXT000000264151, JORFT

## Met en base ces résultats

* Créer la table :
```
create table texte (cid varchar(21), categorie_unique boolean, categorie varchar(21), correspondance_struct_version boolean, struct_version_non_vide boolean, etat varchar(21));
```

* Créer un index :
```
create index on texte (cid);
```


In [140]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

In [141]:
for cid in liste_cid_unique:
    nb_categorie = nb_categorie_par_cid[cid]
    
    if nb_categorie > 1:
        curseur.execute("insert into texte (cid, categorie_unique, etat) values (%s, %s, %s)",
                        (cid, False, 'anomalie'))
        connection.commit()
        
    else:
        categorie = categorie_par_cid[cid][0]
        
        liste_struct = sorted(list(os.listdir(os.path.join(racine_source, categorie, cid, 'struct'))))
        liste_version = sorted(list(os.listdir(os.path.join(racine_source, categorie, cid, 'version'))))
        correspondance_struct_version = (liste_struct == liste_version)
        struct_version_non_vide = (len(liste_struct) > 0) if correspondance_struct_version else None

        curseur.execute("""
            insert into texte (
                cid,
                categorie_unique,
                categorie,
                correspondance_struct_version,
                struct_version_non_vide,
                etat
                ) values (%s, %s, %s, %s, %s, %s)
            """, (
                cid,
                True,
                categorie,
                correspondance_struct_version,
                struct_version_non_vide,
                'valide' if (correspondance_struct_version and struct_version_non_vide) else 'anomalie',
            ))
        connection.commit()


In [142]:
curseur.close()
connection.close()

## Unicité des articles et des section_ta (lorsque categorie_unique)

In [111]:
def trouve_enfants(dossier):
    enfants = os.listdir(dossier)
    est_fichier_liste = [os.path.isfile(os.path.join(dossier, f)) for f in enfants]
    enfants_fichier = [enfants[i] for i in range(len(enfants)) if est_fichier_liste[i]]
    enfants_dossier = [enfants[i] for i in range(len(enfants)) if not est_fichier_liste[i]]
    
    return enfants_fichier, enfants_dossier

In [120]:
liste_fichier = []
liste_fichier_par_cid = {}
for cid in liste_cid_unique:
    if nb_categorie_par_cid[cid] > 1:
        continue
        
    categorie = categorie_par_cid[cid][0]        
    enfants_fichier, enfants_dossier = trouve_enfants(os.path.join(racine_source, categorie, cid))
    
    liste_fichier_par_cid[cid] = enfants_fichier
    liste_fichier += enfants_fichier

In [121]:
len(liste_fichier), len(set(liste_fichier))

(1399475, 1399358)

In [122]:
Counter(Counter(liste_fichier).values())

Counter({1: 1399241, 2: 117})

In [123]:
liste_duplication = [article for article, compte in Counter(liste_fichier).items() if compte > 1]

In [127]:
len(liste_duplication)

117

In [128]:
', '.join(liste_duplication)

'LEGISCTA000006105436.xml, LEGIARTI000018618238.xml, LEGIARTI000006495445.xml, LEGIARTI000006495446.xml, LEGIARTI000006775601.xml, LEGIARTI000006726102.xml, LEGIARTI000006604386.xml, LEGIARTI000006605751.xml, LEGIARTI000006775599.xml, LEGIARTI000006726113.xml, LEGIARTI000006776924.xml, LEGIARTI000006726101.xml, LEGIARTI000006726104.xml, LEGIARTI000006352316.xml, LEGIARTI000024411342.xml, LEGIARTI000006776925.xml, LEGIARTI000022335229.xml, LEGIARTI000006328239.xml, LEGIARTI000006382243.xml, LEGIARTI000006726123.xml, LEGIARTI000006604381.xml, LEGIARTI000006605750.xml, LEGIARTI000006604387.xml, LEGIARTI000006726117.xml, LEGIARTI000006604376.xml, LEGIARTI000006775606.xml, LEGIARTI000006382242.xml, LEGIARTI000028252943.xml, LEGIARTI000006605752.xml, LEGIARTI000019863646.xml, LEGIARTI000006726118.xml, LEGIARTI000006328235.xml, LEGISCTA000006105438.xml, LEGISCTA000006098713.xml, LEGIARTI000006605749.xml, LEGIARTI000006726098.xml, LEGIARTI000006605746.xml, LEGIARTI000006726099.xml, LEGIARTI000

In [129]:
set_duplication = set(liste_duplication)

Ajout de la colonne `non_duplication_article_section_ta` :
```
alter table texte add non_duplication_article_section_ta boolean;
```

In [143]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

In [146]:
liste_cid_duplication = []
for cid in liste_cid_unique:
    if (nb_categorie_par_cid[cid] > 1):
        continue
        
    if set(liste_fichier_par_cid[cid]) & set_duplication:
        liste_cid_duplication.append(cid)
        non_duplication_article_section_ta = False
    else:
        non_duplication_article_section_ta = True
        
    curseur.execute("select etat from texte where cid = %s;", (cid,))
    resultats_requete = curseur.fetchall()
    assert len(resultats_requete) == 1
    etat_vieux = resultats_requete[0][0]
    
    etat = 'valide' if ((etat_vieux == 'valide') and non_duplication_article_section_ta) else 'anomalie'
    
    curseur.execute("update texte set non_duplication_article_section_ta = %s, etat = %s where cid = %s;",
                    (non_duplication_article_section_ta, etat, cid))
    connection.commit()

In [147]:
len(liste_cid_duplication), ', '.join(liste_cid_duplication)

(22,
 'JORFTEXT000000497292, JORFTEXT000000764317, JORFTEXT000000387673, LEGITEXT000006073163, JORFTEXT000000727078, JORFTEXT000000343285, LEGITEXT000006070573, JORFTEXT000000321041, LEGITEXT000006070027, JORFTEXT000000569755, JORFTEXT000000881088, LEGITEXT000006070217, JORFTEXT000000752261, JORFTEXT000000317681, LEGITEXT000006070025, JORFTEXT000000534307, JORFTEXT000000752297, JORFTEXT000000167078, JORFTEXT000000217071, JORFTEXT000000170299, JORFTEXT000000572834, JORFTEXT000000846705')

In [148]:
curseur.close()
connection.close()

95663 cid valides